### Ćelijski automati

Ćelijski automati su klasa modela koji se mogu koristiti za simulaciju različitih prirodnih fenomena. Korišćeni su za modeliranje razvoja tumora, kroz trodimenzionalnu mapu tkiva i predviđanje rasta ćelija kroz vreme. U neuronauci, ćelijski automati se mogu koristi za izučavanje populacije aktivacije neurona. Postoje mnogo druge primene, uključujući one u hemiji, biologiji, ekologiji i mnogim drugim granama nauke.

Ćelijski automati su u osnovi skup (prostorno) uređenih jedinica koje nazivamo ćelije. Svaka ćelija vrši neku od mogućih akcija, ili uzima neko od mogućih stanja, u zavisnosti od svoje neposredne okoline. Možda najpoznatiji primer ćelijskih automata je Konvejeva Igra života.

### Igra života

Konvejeva Igra života je ćelijski automat koji se sastoji iz pravougaone mreže ćelija. Svaka ćeilja može biti u jednom od sva stanja: *živa* i *mrtva*. Igra se odvija kroz vremenske iteracije, gde ćelije u jednoj iteraciji računaju svoje stanje u sledećoj. Stanje ćelije se računa na osnovu svog trenutnog stanja, trenutnog stanja svih neposrednih 8 suseda ćelije i sledećih pravila:
- Ako ćelija ima manje od 2 živa suseda, u sledećoj iteraciji će biti mrtva 
- Ako ćelija ima više od 3 živa suseda, u sledećoj iteraciji će biti mrtva 
- Ako je ćelija živa i ima 2 ili 3 živa suseda, u sledećoj iteraciji će biti živa
- Ako je ćelija mrtva i ima 3 živa suseda, u sledećoj iteraciji će biti živa

Osnovna struktura podataka Igre života je matrica stanja, koja za svaku ćeliju a[i, j] sadrži podatak da li je ćelija živa ili mrtva. Pri implementaciji igre javlja se problem suseda ćelija koje se nalaze uz ivicu mreže (ćelije u prvoj koloni nemaju "leve" susede, itd.). Postoji više rešenja za ovaj problem kao što su:
- Pretpostaviti da su nepostojeći susedi uvek mrtvi
- Uvesti "cikluse", tako da je poslednja kolona "levi" sused prve kolone, a poslednji red "gornji" sused prvog reda (efektivno torus).

Pri izradi zadataka koristiti drugi pristup.

### Zadatak
Zadatak je implementirati paralelizovanu verziju Igre života u programskom jeziku Python, na nekoliko načina. Prilikom implementacije obezbediti da se posle izvršavanja zadatog broja iteracija sačuva niz matrica stanja kroz vreme (stanja sistema u svakoj iteraciji), koji je kompatibilan sa datom funkcijom za animaciju (sledeća ćelija u fajlu).

1. Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom Ključevima, Semaforima i Uslovima (7 poena)  
  Svaka nit simulira rad jedne ćelije u sistemu i ima pristup stanjima svojih suseda. Ćelije nemaju pristup globalnom brojaču iteracija, već svaka ćelija interno vodi računa o broju trenutne iteracije. Pored matrice podataka potrebno je uvesti:
  - Listu brojača suseda koji su pročitali trenutnu vrednost (za svaku ćeliju po jedan brojač). Osmi (poslednji) sused koji pročita vrednost budi ćeliju kako bi mogla da upiše novu vrednost u matricu stanja (buđenje realizovati semaforom). Voditi računa o sinhronizaciji suseda koji menjaju vrednost brojača.
  - Uslov (Condition) na kome čekaju sve ćelije koje su upisale novu vrednost u matricu stanja, pre nego što pređu u sledeću iteraciju. 
  - Brojač ćelija, zaštićen ključem, koje su upisale novu vrednost u matricu stanja. Poslednja ćelija aktivira Uslov za sledeću iteraciju. Voditi računa o redosledu uzimanja i puštanja ključa za brojač i ključa za uslov.
  
2. Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom redovima za poruke (6 poena)  
  Svaka nit simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. Za potrebe analize rada može se uvesti deljeni niz matrica stanja (i-ti element niza je matrica stanja i-te iteracije), u koji ćelije upisuju svoja stanja (ćelije ne mogu čitati iz ovog niza!).
  
3. Upotrebom procesa koji simuliraju *po jednu ćeliju* i sinhronizacijom redovima za poruke (6 poena)  
  Svaki proces je simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. Za potrebe analize rada implementirati poseban servis (dodatni proces) kojem sve ćelije javljaju novo stanje prilikom promene (pri čemu poruke sadrže koordinate ćelije, broj iteracije i novo stanje). Servis treba da rekonstruiše i sačuva (ili vrati u glavni program) niz matrica stanja.

4. Upotrebom više procesa kroz *process pool*, generisanjem taskova na nivou skupa ćelija (6 poena)  
  Matricu stanja podeliti na N delova (gde je konfigurabilni parametar) i za svaki deo generisati *task* (poziv funkcije čijim parametrima se definiše koji deo matrice treba obraditi). Funkcija treba da vrati niz koordinata ćelija i njihove nove vrednosti, a matrica za sledeću iteraciju se može kreirati u glavnom programu. Trenutne vrednosti ćelija i suseda se mogu čitati iz deljene matrice.
  
***Napomena:*** Zadati pristupi 1-3 nisu uobičajen način paralelizacije Igre života, ali su odabrani kao dobar primer za vežbu rada sa paralelnim modelima i tehnikama sinhronizacije.

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np

def animate(steps):
  ''' Prima niz matrica (svaka matrica je stanje u jednom koraku simulacije) 
  prikazuje razvoj sistema'''
  
  def init():
    im.set_data(steps[0])
    return [im]
  
  
  def animate(i):
    im.set_data(steps[i])
    return [im]

  im = plt.matshow(steps[0], interpolation='None', animated=True);
  
  anim = FuncAnimation(im.get_figure(), animate, init_func=init,
                  frames=len(steps), interval=500, blit=True, repeat=False);
  return anim


n = 20
steps = [(np.random.rand(n**2).reshape(n, n) > 0.5).astype(np.int8)for i in range(50)]
anim = animate(steps);
HTML(anim.to_html5_video())

In [ ]:
import numpy as np

In [ ]:
a = np.zeros((20,5))

In [ ]:
a[3,3]

0.0

In [ ]:
#UVOD

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

# Setting up the values for the grid
ALIVE = 255
DEAD = 0
values = [ALIVE, DEAD]


def random_grid(n):
    """
    Returns a grid of NxN random values.
    :param n: Size of the grid.
    :return: A grid of NxN random values.
    """
    return np.random.choice(values, n * n).reshape(n, n)


def update(frame_number, img, grid, n):
    """
    Updates given grid.
    :param frame_number: Current frame.
    :param img: Current image.
    :param grid: Current matrix.
    :param n: Size of the grid.
    :return: Display an image, i.e. data on a 2D regular raster.
    """

    # Copy grid
    new_grid = grid.copy()

    for i in range(n):
        for j in range(n):
            # Compute 8-neighbour sum using toroidal boundary conditions
            total = int((grid[i, (j - 1) % n] + grid[i, (j + 1) % n] +
                         grid[(i - 1) % n, j] + grid[(i + 1) % n, j] +
                         grid[(i - 1) % n, (j - 1) % n] + grid[(i - 1) % n, (j + 1) % n] +
                         grid[(i + 1) % n, (j - 1) % n] + grid[(i + 1) % n, (j + 1) % n]) / 255)

            # Conway's rules
            if grid[i, j] == ALIVE:
                if (total < 2) or total > 3:
                    new_grid[i, j] = DEAD
            else:
                if total == 3:
                    new_grid[i, j] = ALIVE

    # Update grid
    img.set_data(new_grid)
    grid[:] = new_grid[:]
    return img


def play_game():
    """
    Simulates Conway's Game of Life.
    """

    # Grid size
    N = 50
    FRAMES = 10
    UPDATE_INTERVAL = 50
    SAVE_COUNT = 50

    grid = random_grid(N)

    for i in range(N):
        for j in range(N):
            # 3x3 Matrix of 8 neighbours of the current cell
            matrix = [[
                ((i - 1) % N, (j - 1) % N),  # upper left
                ((i - 1) % N, j),  # upper
                ((i - 1) % N, (j + 1) % N)  # upper right
            ], [
                (i, (j - 1) % N),  # left
                (i, j),  # center
                (i, (j + 1) % N)  # right
            ], [
                ((i + 1) % N, (j - 1) % N),  # lower left
                ((i + 1) % N, j),  # lower
                ((i + 1) % N, (j + 1) % N)  # lower right
            ]]

            print('\nMatrix for ({}, {}):'.format(i, j))

            for k in range(3):
                for v in range(3):
                    print(matrix[k][v], end='')
                print()

    fig, ax = plt.subplots()
    img = ax.imshow(grid, interpolation='nearest')
    anim = animation.FuncAnimation(
        fig,
        update,
        fargs=(img, grid, N,),
        frames=FRAMES,
        interval=UPDATE_INTERVAL,
        save_count=SAVE_COUNT
    )
    plt.show()


if __name__ == '__main__':
    play_game()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import threading
from threading import Thread
import random
import time

N = 20
ON = 255
OFF = 0
vals = [ON,OFF]
#randomGrid
grid = np.random.choice(vals, N * N, p=[0.2, 0.8]).reshape(N, N)
listaMatrica = []

plt.imshow(grid, interpolation='nearest')
plt.show()

cellsFinished = 0
numOfCellsFinished = threading.Lock()
accessListiBrojaca = threading.Lock()
nextIteration = threading.Condition()

class Celija(Thread):

    def __init__(self, row, column, state):
        super().__init__()
        self.currentIteration = 0
        self.row = row
        self.column = column
        self.state = state
        self.listaBrojacaSuseda = [0, 0, 0, 0, 0, 0, 0, 0]
        self.semaphore = threading.Semaphore(0)

    def checkNeighbors(self):

        total = 0
        for i in range(0, 8):
            if (i == 0):
                # 1
                # levi sused
                # posecen od strane desnog
                total += int(grid[self.row, (self.column - 1) % N])

                # zakljucava pristup listi brojaca suseda kako bi promenio vrednost

                for t in threads:
                    if t.row == self.row and t.column == (self.column - 1) % N:
                        t.listaBrojacaSuseda[i] += 1
                        # TODO proveri koja je vrednost u listi nakon ovoga


                    # print("Uvecam brojac za levi sused")

                    t.semaphore.release()

            if (i == 1):
                # 2
                # desni sused
                # posecen od strane levog
                total += int(grid[self.row, (self.column + 1) % N])


                for t in threads:
                    if t.row == self.row and t.column == (self.column + 1) % N:
                        t.listaBrojacaSuseda[i] += 1

                    # print("Uvecam brojac za desni sused")

                    t.semaphore.release()

            if (i == 2):
                # 3
                # gornji sused
                # posecen od strane donjeg
                total +=int(grid[(self.row - 1) % N, self.column])


                for t in threads:
                    if (t.row == (self.row - 1) % N and t.column == self.column):
                        t.listaBrojacaSuseda[i] += 1

                    # print("Uvecam brojac za gornji sused")

                    t.semaphore.release()


            if (i == 3):
                # 4
                # donji sused
                # posecen od strane gornjeg
                total += int(grid[(self.row + 1) % N, self.column])


                for t in threads:
                    if (t.row == (self.row + 1) % N and t.column == self.column):
                        t.listaBrojacaSuseda[i] += 1
                    # print("Uvecam brojac za donji sused")

                    t.semaphore.release()


            if (i == 4):
                # 5
                # gornji levi sused
                # posecen od stranje donjeg desnog
                total += int(grid[(self.row - 1) % N, (self.column - 1) % N])


                for t in threads:
                    if (t.row == (self.row - 1) % N and t.column == (self.column - 1) % N):
                        t.listaBrojacaSuseda[i] += 1

                    # print("Uvecam brojac za gornji levi sused")

                    t.semaphore.release()


            if (i == 5):
                # 6
                # gornji desni sused
                # posecen od strane donjeg levog
                total += int(grid[(self.row - 1) % N, (self.column + 1) % N])


                for t in threads:
                    if (t.row == (self.row - 1) % N and t.column == (self.column + 1) % N):
                        t.listaBrojacaSuseda[i] += 1
                    #print("Uvecam brojac za gornji desni sused")

                    t.semaphore.release()


            if (i == 6):
                # 7
                # donji levi sused
                # posecen od strane gornjeg desnog
                total += int(grid[(self.row + 1) % N, (self.column - 1) % N])


                for t in threads:
                    if (t.row == (self.row + 1) % N and t.column == (self.column - 1) % N):
                        t.listaBrojacaSuseda[i] += 1
                    # print("Uvecam brojac za donji levi sused")

                    t.semaphore.release()


            if (i == 7):
                # 8
                # donji desni sused
                # posecen od strane gornjeg levog
                total += int(grid[(self.row + 1) % N, (self.column + 1) % N])


                for t in threads:
                    if (t.row == (self.row + 1) % N and t.column == (self.column + 1) % N):
                        t.listaBrojacaSuseda[i] += 1
                    # print("Uvecam brojac za donji desni sused")

                    t.semaphore.release()

        return total / 255

    def update(self, total):
        global cellsFinished
        global grid

        if grid[self.row, self.column] == ON:

            if (total < 2) or (total > 3):
                grid[self.row, self.column] = OFF
        else:
            if total == 3:
                grid[self.row, self.column] = ON

        # zakljucavanje pre povecanja broja celija koje su zavrsile za iteracijom
        numOfCellsFinished.acquire()
        # print("Menjam vrednost jer sam zavrsio")
        # print("Broj celija koje su zavrsile1:" + str(cellsFinished))
        cellsFinished += 1
        numOfCellsFinished.release()

        self.currentIteration += 1

    def run(self):
        global cellsFinished
        # TODO read value from neighbors
        for o in range(0,2):
            total = self.checkNeighbors()

            # print("Total " + str(self.row) + " " + str(self.column) + " : " + str(total))
            # waiting for everyone to read(waiting for semaphore to have value 1)
            # print("Brojac za semafor: "+ str(self.brojacZaSemafor))
            # print("Vrednost brojaca: "+str(brojac))
            while True:
                self.semaphore.acquire()

                # print("Celija "+str(self.row)+str(self.column))
                accessListiBrojaca.acquire()
                #if self.listaBrojacaSuseda[0] == 1 and self.listaBrojacaSuseda[1] == 1 and self.listaBrojacaSuseda[1] == 1 and self.listaBrojacaSuseda[3] == 1 and self.listaBrojacaSuseda[4] == 1  and self.listaBrojacaSuseda[5] == 1 and self.listaBrojacaSuseda[6] == 1 and self.listaBrojacaSuseda[7] == 1:
                if all(x > 0 for x in self.listaBrojacaSuseda):    
                    for i in range(0,8):
                        self.listaBrojacaSuseda[i] = 0
                    accessListiBrojaca.release()
                    break
                accessListiBrojaca.release()

            timeOfSleep = random.random()
            # print("Vreme spavanja :" + str(timeOfSleep))
            time.sleep(timeOfSleep)
            # TODO write value based on neighbors
            self.update(total)
            numOfCellsFinished.acquire()
            if(cellsFinished == N*N):
                # print("Celija " + str(self.row) + str(self.column)+" radi notifyAll")
                # print("RADIM NotifyAll")
                cellsFinished = 0
                numOfCellsFinished.release()
                nextIteration.acquire()
                nextIteration.notifyAll()
                nextIteration.release()
                print("Crtam grid",self.currentIteration,o)

                listaMatrica.append(grid.copy())

            else:
                numOfCellsFinished.release()
                nextIteration.acquire()
                nextIteration.wait()
                nextIteration.release()




threads = []
# Starting threads
for i in range(0, N):
    for j in range(0, N):
        # print("Starting thread: " + i.__str__() + j.__str__())
        thread = Celija(i, j, grid[i][j])
        threads.append(thread)

for t in threads:
    t.start()

for t in threads:
    t.join()

for g in listaMatrica:
    plt.imshow(g, interpolation='nearest')
    plt.show()

In [ ]:
#Upotrebom niti koje simuliraju po jednu ćeliju i sinhronizacijom redovima za poruke (6 poena)
#Svaka nit simulira rad jedne ćelije u sistemu. 
#Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). 
#Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. 
#Za potrebe analize rada može se uvesti deljeni niz matrica stanja (i-ti element niza je matrica stanja i-te iteracije),
#u koji ćelije upisuju svoja stanja (ćelije ne mogu čitati iz ovog niza!).

import threading
import queue
from threading import Thread
import random
import time
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np

# todo da li mogu da procitam prvo stanje iz matrice? - moze
# todo svaka celija ima svoj queue za poruke
# todo susedima javljam svoje stanje(nije blokirajuce)
# todo celija je blokirana nad queue-om dok svaka celija ne javi stanje(8 puta radim wait nad queue)
# todo svaka celija ceka da joj svi susedi kazu svoja stanja
# todo prodjem kroz queue i update svoje stanje na osnovu njih
# todo da li je dozvoljeno upisati stanje u matricu kad zavrsim update?


N = 20
ON = 255
OFF = 0
vals = [ON, OFF]
# randomGrid
grid = np.random.choice(vals, N * N, p=[0.2, 0.8]).reshape(N, N)

listaMatrica = []

#plt.imshow(grid, interpolation='nearest')
#plt.show()

cellsFinished = 0
numOfCellsFinished = threading.Condition()


def animate(listaMatrica):
  ''' Prima niz matrica (svaka matrica je stanje u jednom koraku simulacije) 
  prikazuje razvoj sistema'''
  
  def init():
    im.set_data(listaMatrica[0])
    return [im]
  
  
  def animate(i):
    im.set_data(listaMatrica[i])
    return [im]

  im = plt.matshow(listaMatrica[0], interpolation='None', animated=True);
  
  anim = FuncAnimation(im.get_figure(), animate, init_func=init,
                  frames=len(listaMatrica), interval=500, blit=True, repeat=False);
  return anim

class Celija(Thread):

    def __init__(self, row, column, state):
        super().__init__()
        self.currentIteration = 0
        self.row = row
        self.column = column
        self.state = state
        self.neighborsQueue = queue.Queue()

    def signalNeighbors(self):
        for t in threads:
            if t.row == self.row and t.column == (self.column - 1) % N:
                t.neighborsQueue.put(self.state)
            if t.row == self.row and t.column == (self.column + 1) % N:
                t.neighborsQueue.put(self.state)
            if t.row == (self.row - 1) % N and t.column == self.column:
                t.neighborsQueue.put(self.state)
            if t.row == (self.row + 1) % N and t.column == self.column:
                t.neighborsQueue.put(self.state)
            if t.row == (self.row - 1) % N and t.column == (self.column - 1) % N:
                t.neighborsQueue.put(self.state)
            if t.row == (self.row - 1) % N and t.column == (self.column + 1) % N:
                t.neighborsQueue.put(self.state)
            if t.row == (self.row + 1) % N and t.column == (self.column - 1) % N:
                t.neighborsQueue.put(self.state)
            if t.row == (self.row + 1) % N and t.column == (self.column + 1) % N:
                t.neighborsQueue.put(self.state)
            
    def update(self, total):
        global cellsFinished
        global grid

        if self.state == ON:

            if (total < 2) or (total > 3):
                self.state = OFF
                grid[self.row, self.column] = OFF
        else:
            if total == 3:
                self.state = ON
                grid[self.row, self.column] = ON

        # zakljucavanje pre povecanja broja celija koje su zavrsile za iteracijom
        numOfCellsFinished.acquire()
        cellsFinished += 1
        numOfCellsFinished.release()

        self.currentIteration += 1

    def run(self):
        global cellsFinished
        # TODO read value from neighbors
        for o in range(0, 10):
            self.signalNeighbors()

            total = 0
            for i in range(0, 8):
                total += self.neighborsQueue.get()

            timeOfSleep = random.random()

            time.sleep(timeOfSleep)

            self.update(total / 255)

            numOfCellsFinished.acquire()
            if cellsFinished == N * N:
                # print("skrt")
                cellsFinished = 0
                numOfCellsFinished.notifyAll()
                numOfCellsFinished.release()
                listaMatrica.append(grid.copy())
            else:
                numOfCellsFinished.wait()
                numOfCellsFinished.release()


threads = []
# Starting threads
for i in range(0, N):
    for j in range(0, N):
        # print("Starting thread: " + i.__str__() + j.__str__())
        thread = Celija(i, j, grid[i][j])
        threads.append(thread)

for t in threads:
    t.start()

for t in threads:
    t.join()

anim = animate(listaMatrica);
HTML(anim.to_html5_video())

Za potrebe analize rada može se uvesti deljeni niz matrica stanja (i-ti element niza je matrica stanja i-te iteracije), u koji ćelije upisuju svoja stanja (ćelije ne mogu čitati iz ovog niza!).

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import multiprocessing
from multiprocessing import Process, Value
import random
import time

# todo da li mogu da procitam prvo stanje iz matrice? - moze
# todo svaka celija ima svoj queue za poruke
# todo susedima javljam svoje stanje(nije blokirajuce)
# todo celija je blokirana nad queue-om dok svaka celija ne javi stanje(8 puta radim wait nad queue)
# todo svaka celija ceka da joj svi susedi kazu svoja stanja
# todo prodjem kroz queue i update svoje stanje na osnovu njih
# todo da li je dozvoljeno upisati stanje u matricu kad zavrsim update?


N = 20
ON = 255
OFF = 0
vals = [ON, OFF]
# randomGrid
grid = np.random.choice(vals, N * N, p=[0.2, 0.8]).reshape(N, N)

plt.imshow(grid, interpolation='nearest')
plt.show()

cellsFinished = Value('i', 0)
numOfCellsFinished = multiprocessing.Condition()


class Celija(Process):
    def __init__(self, row, column, state, gridSize, serviceQueue, matrixQueueCell, numOfCellsFinished,cellsFinished,valueOn,valueOff):
        super().__init__()
        self.currentIteration = 0
        self.row = row
        self.column = column
        self.state = state
        self.gridSize = gridSize
        self.serviceQueue = serviceQueue
        self.matrixQueueCell = matrixQueueCell
        self.numOfCellsFinished = numOfCellsFinished
        self.cellsFinished = cellsFinished
        self.valueOn = valueOn
        self.valueOff = valueOff

    def signalNeighbors(self):

        self.matrixQueueCell[self.row][(self.column - 1) % self.gridSize].put(self.state)

        self.matrixQueueCell[self.row][(self.column + 1) % self.gridSize].put(self.state)

        self.matrixQueueCell[(self.row - 1) % self.gridSize][self.column].put(self.state)

        self.matrixQueueCell[(self.row + 1) % self.gridSize][self.column].put(self.state)

        self.matrixQueueCell[(self.row - 1) % self.gridSize][(self.column - 1) % self.gridSize].put(self.state)

        self.matrixQueueCell[(self.row - 1) % self.gridSize][(self.column + 1) % self.gridSize].put(self.state)

        self.matrixQueueCell[(self.row + 1) % self.gridSize][(self.column - 1) % self.gridSize].put(self.state)

        self.matrixQueueCell[(self.row + 1) % self.gridSize][(self.column + 1) % self.gridSize].put(self.state)

    def update(self, total):
        if self.state == self.valueOn:
            if (total < 2) or (total > 3):
                self.state = self.valueOff
        else:
            if total == 3:
                self.state = self.valueOn


        # with self.cellsFinished.get_lock():
        #     # print("Cell finished ",cellsFinished.value)
        #     self.cellsFinished.value += 1

        self.currentIteration += 1
        cellInfo = (self.row, self.column, self.currentIteration, self.state)
        # print('cellsFinished:', cellsFinished.value)
        self.serviceQueue.put(cellInfo)
        # todo spakuj koordinate, broj iteracije i novo stanje u tuple i onda odradi put u queue

    def run(self):

        for i in range(0,5):

            self.signalNeighbors()
            total = 0

            for i in range(0, 8):
                total += self.matrixQueueCell[self.row][self.column].get()

            # timeOfSleep = random.random()
            # time.sleep(timeOfSleep)
            # print("Total",total // 255)
            self.update(total // 255)
            self.numOfCellsFinished.acquire()

            self.cellsFinished.get_lock().acquire()
            # print("Cell finished ",cellsFinished.value)
            self.cellsFinished.value += 1

            if self.cellsFinished.value == self.gridSize * self.gridSize:
                self.cellsFinished.value = 0
                self.cellsFinished.get_lock().release()
                print("clear cells finished:", cellsFinished.value,self.currentIteration)
                self.numOfCellsFinished.notify_all()
                self.numOfCellsFinished.release()
            else:
                self.cellsFinished.get_lock().release()
                # print("Celija", self.row, self.column, "ceka")
                self.numOfCellsFinished.wait()
                # print("Celija", self.row, self.column, "izasao")
                self.numOfCellsFinished.release()


class ServiceProces(Process):
    def __init__(self, grid, gridSize, serviceQueue,listaMatrica):
        super().__init__()
        self.grid = grid
        self.gridSize = gridSize
        self.serviceQueue = serviceQueue
        self.listaMatrica = listaMatrica

    def run(self):

        for i in range(0,5):

            for i in range(0, self.gridSize * self.gridSize):
                cellsInfo = self.serviceQueue.get()
                # print(i, '- Cells info:', cellsInfo)
                self.grid[cellsInfo[0]][cellsInfo[1]] = cellsInfo[3]

            self.listaMatrica.append(self.grid.copy())

        for g in listaMatrica:
            plt.imshow(g, interpolation='nearest')
            plt.show()



serviceQueue = multiprocessing.Queue()



matrixQueueCell = [[multiprocessing.Queue() for i in range(N)] for j in range(N)]

listaMatrica= []

serviceProcess = ServiceProces(grid, N, serviceQueue,listaMatrica)
serviceProcess.start()

# Starting threads
# for i in range(0, N):
#     for j in range(0, N):
# print("Starting thread: " + i.__str__() + j.__str__())
# process = Celija(i, j, grid[i][j], N, serviceQueue, matrixQueueCell, numOfCellsFinished, lock)
process = [[Celija(i, j, grid[i][j], N, serviceQueue, matrixQueueCell, numOfCellsFinished, cellsFinished,ON,OFF) for i in range(N)] for j in
           range(N)]

for t in process:
    for op in t:
        op.start()

for t in process:
    for op in t:
        op.join()

serviceProcess.join()

for g in listaMatrica:
    plt.imshow(g, interpolation='nearest')
    plt.show()

# todo queue-ve,lock,condition,broj itracija,velicina matrice,
# todo pracenje koliko je celija zavrsilo, za to koristiti value
# todo kontrol procesu se salje broj iteracija,matricu,velicini grida,service queue(celije komuniciraju sa service procesom),
# todo retrun queue(ne mora) ali da bi napravio animaciju mora da se pozove anime iz main-a
# todo mogu da mu prosledim matricu na pocetku i on ce nju da menja

In [ ]:
# TODO: Task 4
# TODO: =============================================
# TODO: recimo imam 5x5 matricu i pravim 5 procesa za svaki red u matrici
# TODO: glavni program spawnuje procese koji rade analizu rada
# TODO: glavni program ima celu matricu i dace procesu sve potrebne vrednosti svih suseda
# TODO: funkcija je zapravo podproces koja ce da vrati nazad glavnom programu
# TODO: Process Pool -> map() i apply()
# TODO: damo matricu i on nam vrati nazad
# TODO: funkc koju prosledim poolu da vrati i, j i novu vrednost
# TODO: =============================================

import numpy as np
import matplotlib.pyplot as plt
import json
from multiprocessing import Pool, cpu_count


ITERATIONS = 20
N = 20
ALIVE = 255
DEAD = 0
values = [ALIVE, DEAD]
grid = np.random.choice(values, N ** 2, p=(0.2, 0.8)).reshape(N, N)


def play_game(task, grid, N):
    coordinates = {}

    for x, y in task:
        result = {
            'upper_left': grid[(x - 1) % N, (y - 1) % N],
            'upper': grid[(x - 1) % N, y],
            'upper_right': grid[(x - 1) % N, (y + 1) % N],
            'left': grid[x, (y - 1) % N],
            # 'center': grid[x, y],
            'right': grid[x, (y + 1) % N],
            'lower_left': grid[(x + 1) % N, (y - 1) % N],
            'lower': grid[(x + 1) % N, y],
            'lower_right': grid[(x + 1) % N, (y + 1) % N]
        }

        # print(json.dumps(result, default=str, indent=4))

        total = 0

        for v in result.values():
            total += v

        total //= 255

        # Conway's rules
        if grid[x, y] == ALIVE:
            if total < 2 or total > 3:
                coordinates[(x, y)] = DEAD
            else:
                coordinates[(x, y)] = ALIVE
        else:
            if total == 3:
                coordinates[(x, y)] = ALIVE
            else:
                coordinates[(x, y)] = DEAD

    return coordinates


if __name__ == '__main__':
    matrix_list = [grid.copy()]
    tasks = []
    results = []

    print(grid)
    print()

    # Add grid rows
    for i in range(N):
        row = []
        for j in range(N):
            row.append((i, j))
        tasks.append(row)

    pool = Pool(cpu_count())

    for i in range(ITERATIONS):
        results = [pool.apply(play_game, args=(task, grid, N,)) for task in tasks]
        # results.append(result)
        # print('results:', results)

        for r in results:
            for k, v in r.items():
                grid[k[0], k[1]] = v

        matrix_list.append(grid.copy())

    pool.close()
    pool.join()

    for matrix in matrix_list:
        plt.imshow(matrix, interpolation='nearest')
        plt.show()